# Testing for modules impacted by diet

In [1]:
#load packages
pkgs <- c("mgcv", "lme4", "ggplot2", "vroom", "dplyr", "forcats", "tidyr", "Seurat", "SingleCellExperiment", "gratia", "MASS", "fitdistrplus", "furrr", "tidyverse")
vapply(pkgs, library, logical(1), character.only = TRUE, logical.return = TRUE,
       quietly = TRUE)

This is mgcv 1.8-42. For overview type 'help("mgcv-package")'.


Attaching package: ‘lme4’


The following object is masked from ‘package:nlme’:

    lmList



Attaching package: ‘dplyr’


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack


The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in pl

mgcv                 lme4              ggplot2 
                TRUE                 TRUE                 TRUE 
               vroom                dplyr              forcats 
                TRUE                 TRUE                 TRUE 
               tidyr               Seurat SingleCellExperiment 
                TRUE                 TRUE                 TRUE 
              gratia                 MASS         fitdistrplus 
                TRUE                 TRUE                 TRUE 
               furrr            tidyverse 
                TRUE                 TRUE

In [2]:
# settings for the run
me_dir <- 'wgcna_output/renamed_tables/' #where to find the WGCNA MEs
run_name <- 'ds4_mcs20_pF_kME_cellModEmbed_24h' #the run prefix
group_col <- "diet" # group that will be modelled
cluster_col <- "region" # sample region
mgcv_dir <- paste0(me_dir, 'mgcv/') # where to save the mgcv results
dir.create(mgcv_dir, recursive = TRUE) # creates the mgcv results folder if it doesn't exist
summaries_path <- paste0(mgcv_dir, run_name, '_summaries_f.rds') #save summaries here
models_path <- paste0(mgcv_dir, run_name, '_models.rds') #save models here
memfs_path <- paste0(mgcv_dir, run_name, '_memfs.rds') #save intermediate memfs objects here (only for debugging)
significant_modules <- paste0(mgcv_dir, run_name, '_significant_modules.csv')#where to save the table summarizing significant modules

Warning message in dir.create(mgcv_dir, recursive = TRUE):
“'wgcna_output/renamed_tables/mgcv' already exists”


In [3]:
me = read_csv(paste0(me_dir, 'kME_cellModEmbed.csv')) #read the MEs and inspect them
me %>% head

Rows: 144 Columns: 386
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (3): data, region, sample
dbl (383): HPF__01, HPF__02, HPF__03, HPF__04, HPF__05, HPF__06, HPF__07, HP...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


data,region,sample,HPF__01,HPF__02,HPF__03,HPF__04,HPF__05,HPF__06,HPF__07,⋯,ZI__39,ZI__40,ZI__41,ZI__42,ZI__43,ZI__44,ZI__45,ZI__46,ZI__47,ZI__48
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dmb,HY,S0010_01_chow_HY,0.05197933,0.03023810,0.1824172,0.1229980,-0.05775205,0.019439584,-0.06125762,⋯,-0.08595527,0.05252190,-0.05822008,0.06567769,0.036785185,0.0102144634,0.10537119,0.11220870,-0.04791984,0.020875004
dmb,HY,S0010_02_chow_HY,0.08876762,0.08017922,0.1459963,0.1170462,-0.03252780,-0.004745673,-0.05191100,⋯,-0.11281907,0.02927096,-0.02412367,0.02603893,0.007287426,-0.0017344272,0.10817653,0.11961064,-0.04316019,0.006892997
dmb,HY,S0010_03_chow_HY,0.07109426,0.02853630,0.1832191,0.1384336,-0.06336754,-0.032457115,-0.06025051,⋯,-0.10252465,0.06811403,-0.04506138,0.01064958,0.021634418,-0.0002412614,0.07374108,0.10598614,-0.02872247,0.025098740
dmb,HY,S0010_04_chow_HY,0.02948085,0.06371331,0.1535754,0.1107229,-0.03764149,0.025134355,-0.05945944,⋯,-0.05751049,0.03972461,-0.01765908,0.05303416,-0.004217487,0.0049177488,0.06565093,0.01816232,-0.03737867,0.014236332
dmb,HY,S0010_05_chow_HY,0.11411558,0.03807216,0.1615871,0.1345847,-0.06702112,0.007702094,-0.05492529,⋯,-0.15425467,0.03670603,-0.01827709,0.01769293,-0.020943932,-0.0073797921,0.09252465,0.09507013,-0.02601267,0.019362943
dmb,HY,S0010_06_chow_HY,0.02021261,0.07189122,0.1562818,0.1019575,-0.03508683,-0.025868732,-0.07505565,⋯,-0.09517898,0.05157318,-0.02545144,0.02872648,-0.025390663,0.0255168968,0.07634631,0.05995981,-0.04782112,0.008990742


In [4]:
meta = read_csv('vsd_meta.csv') # read the metadata and reformat some columns
meta = meta %>%
mutate(diet = factor(diet, levels=c('chow', 'HFD', 'fast'))) %>%
mutate(animal_id = as.character(animal_id)) %>%
mutate(region = factor(region, levels=c("HY", 'HPF', 'SC', 'IC', 'PFC', 'ZI'))) %>%
filter(seq_run %in% c('0037')) %>% #filter down to 24h study
mutate(seq_run = factor(seq_run, levels=c('0037'))) 

meta %>% head

Rows: 144 Columns: 26
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (9): name, filename, seq_run, seq_sample, diet, region, hemisphere, no...
dbl  (15): rna_conc_qbit, box_id, animal_id, diet_kcal, experiment_duration,...
dttm  (2): experiment_start, time_of_euthanasia

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


name,filename,seq_run,seq_sample,rna_conc_qbit,box_id,animal_id,diet,diet_kcal,region,⋯,bw_f,bw_change,bw_change_pct,bw_change_pct_ph,fw_i,fw_f,kcal_intake,power_intake,notes,qc
<chr>,<chr>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
S0037_1431_chow_HY,143_1_Hyp_chow_S7,0037,S7,330.0,1431,1431,chow,3227,HY,⋯,23.0,0.3,0.013215859,0.0005506608,663.5,660.1,10971.8,457.1583,NA,ok
S0037_1432_chow_HY,143_2_Hyp_chow_S25,0037,S25,312.0,1432,1432,chow,3227,HY,⋯,25.5,0.4,0.015936255,0.0006640106,634.2,630.0,13553.4,564.7250,NA,ok
S0037_1442_chow_HY,144_2_Hyp_chow_S2,0037,S2,233.4,1442,1442,chow,3227,HY,⋯,28.4,-0.1,-0.003508772,-0.0001461988,656.7,652.4,13876.1,578.1708,NA,ok
S0037_1452_chow_HY,145_2_Hyp_chow_S41,0037,S41,92.4,1452,1452,chow,3227,HY,⋯,26.7,0.1,0.003759398,0.0001566416,683.0,679.5,11294.5,470.6042,NA,ok
S0037_1423_chow_HY,142_3_Hyp_chow_S38,0037,S38,366.0,1423,1423,chow,3227,HY,⋯,25.8,-0.1,-0.003861004,-0.0001608752,593.1,589.1,12908.0,537.8333,Sup. Colliculus for 142.3 is in tube 145.4,ok
S0037_1443_chow_HY,144_3_Hyp_chow_S40,0037,S40,171.0,1443,1443,chow,3227,HY,⋯,24.8,0.0,0.000000000,0.0000000000,642.8,638.7,13230.7,551.2792,NA,ok


In [5]:
# filters the meta down
# meta is the meta dataframe
# group_col is the perturbation column e.g. - Diet
# groups is a vector with conditions for comparison e.g. - c("chow", "HFD")
# cluster_col is the column encoding clusters in the meta. e.g. - region
# clusters is a vector with clusters to filter to. e.g. - HY
filter_meta <- function (meta, group_col, groups, cluster_col, clusters){
    meta_f <- meta %>% 
        filter(
            get(group_col) %in% groups,
            get(cluster_col) %in% clusters
        )
    return (meta_f)
}

In [6]:
make_sp <- function (me_df) {
    sp <- stringr::str_split_fixed(colnames(me)[4:length(colnames(me))], pattern='__', n=2)
    sp <- as.data.frame(sp)
    sp <- apply(sp, 1, function(x) as.vector(c(x[1], x[2], paste0(x[1], '__', x[2]))))
    sp <- t(sp)
    sp <- as.data.frame(sp)
    colnames(sp)[colnames(sp) == 'V1'] <- 'cluster'
    colnames(sp)[colnames(sp) == 'V2'] <- 'module'
    colnames(sp)[colnames(sp) == 'V3'] <- 'ME_col'
    return (sp)
}

In [7]:
# a simple model
# models the diet (group_col)
fit_mgcv <- function(memf){
    model <- bam(ME ~ get(group_col),
                 data = memf, method = "fREML", discrete = T
                )
    return (model)
}


# a more complex model
# accounts for sequencing run as a random effect
# fit_mgcv <- function(memf){
#     model <- bam(ME ~ get(group_col) +
#                  s(seq_run, bs = "re"), # could also model cages if known
#                  data = memf, method = "fREML", discrete = T
#                 )
#     return (model)
# }

safe_fit_mgcv <- purrr::safely(fit_mgcv) #necessary for parallelization, in case we hit errors

In [8]:
# this is what make_sp produces
sp <- make_sp(me)
head(sp)

,cluster,module,ME_col
,<chr>,<chr>,<chr>
1,HPF,01,HPF__01
2,HPF,02,HPF__02
3,HPF,03,HPF__03
4,HPF,04,HPF__04
5,HPF,05,HPF__05
6,HPF,06,HPF__06


In [9]:
#this is what filter_meta produces
filter_meta(meta, "diet", c('chow', 'fast'), "region", c("HY"))

name,filename,seq_run,seq_sample,rna_conc_qbit,box_id,animal_id,diet,diet_kcal,region,⋯,bw_f,bw_change,bw_change_pct,bw_change_pct_ph,fw_i,fw_f,kcal_intake,power_intake,notes,qc
<chr>,<chr>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
S0037_1431_chow_HY,143_1_Hyp_chow_S7,0037,S7,330.0,1431,1431,chow,3227,HY,⋯,23.0,0.3,0.013215859,0.0005506608,663.5,660.1,10971.8,457.1583,NA,ok
S0037_1432_chow_HY,143_2_Hyp_chow_S25,0037,S25,312.0,1432,1432,chow,3227,HY,⋯,25.5,0.4,0.015936255,0.0006640106,634.2,630.0,13553.4,564.7250,NA,ok
S0037_1442_chow_HY,144_2_Hyp_chow_S2,0037,S2,233.4,1442,1442,chow,3227,HY,⋯,28.4,-0.1,-0.003508772,-0.0001461988,656.7,652.4,13876.1,578.1708,NA,ok
S0037_1452_chow_HY,145_2_Hyp_chow_S41,0037,S41,92.4,1452,1452,chow,3227,HY,⋯,26.7,0.1,0.003759398,0.0001566416,683.0,679.5,11294.5,470.6042,NA,ok
S0037_1423_chow_HY,142_3_Hyp_chow_S38,0037,S38,366.0,1423,1423,chow,3227,HY,⋯,25.8,-0.1,-0.003861004,-0.0001608752,593.1,589.1,12908.0,537.8333,Sup. Colliculus for 142.3 is in tube 145.4,ok
S0037_1443_chow_HY,144_3_Hyp_chow_S40,0037,S40,171.0,1443,1443,chow,3227,HY,⋯,24.8,0.0,0.000000000,0.0000000000,642.8,638.7,13230.7,551.2792,NA,ok
S0037_1424_fast_HY,142_4_Hyp_fast_S53,0037,S53,497.0,1424,1424,fast,0,HY,⋯,20.2,-4.9,-0.195219124,-0.0081341301,0.0,0.0,0.0,0.0000,NA,ok
S0037_1425_fast_HY,142_5_Hyp_fast_S69,0037,S69,300.0,1425,1425,fast,0,HY,⋯,20.9,-4.3,-0.170634921,-0.0071097884,0.0,0.0,0.0,0.0000,NA,ok
S0037_1433_fast_HY,143_3_Hyp_fast_S23,0037,S23,168.6,1433,1433,fast,0,HY,⋯,20.5,-5.1,-0.199218750,-0.0083007812,0.0,0.0,0.0,0.0000,Slightly apathetic,ok


In [10]:
# creating splits of the data for parallelization

splits <- list(c('chow', 'fast'))

memfs <- c()

for (s in splits){
    for (i in 1:nrow(sp)){
        cluster <- sp[i,]$cluster
        module <- sp[i,]$module
        me_col <- sp[i,]$ME_col
        me_keep_cols <- c('sample', me_col)
        me_f <- me[me_keep_cols]
        colnames(me_f)[2] <- 'ME'
        mf <- filter_meta(meta, group_col, s, cluster_col, c(cluster))
        mf$sample <- mf$name #need to give this same column as data
        mf_cols <- c('sample', 'diet', 'region', 'seq_run') 
        mf_f <- mf[mf_cols]
        mf_f <- mf_f %>%
            mutate(
                diet = droplevels(diet),
                region = droplevels(region),
                seq_run = droplevels(seq_run)
            )
        memf <- plyr::join(me_f, mf_f, by = c('sample'), type='right')
        memf_name <- paste0(s[1], '_', s[2], '-', me_col)
        memfs[[memf_name]] <- memf
    }
}

In [11]:
# what the splicts look like
memfs[1]

sample,ME,diet,region,seq_run
<chr>,<dbl>,<fct>,<fct>,<fct>
S0037_1422_chow_HPF,-0.02005092,chow,HPF,0037
S0037_1432_chow_HPF,-0.09387324,chow,HPF,0037
S0037_1442_chow_HPF,-0.04276030,chow,HPF,0037
S0037_1423_chow_HPF,0.09008606,chow,HPF,0037
S0037_1443_chow_HPF,-0.04758924,chow,HPF,0037
S0037_1433_fast_HPF,-0.06603858,fast,HPF,0037
S0037_1434_fast_HPF,0.04890475,fast,HPF,0037
S0037_1435_fast_HPF,-0.41309828,fast,HPF,0037
S0037_1445_fast_HPF,0.10889821,fast,HPF,0037


In [12]:
# saveRDS(memfs, memfs_path)

In [13]:
# This does run in parallel! adjust workers accordingly
plan(multisession, workers = 20)
models <- future_map(memfs, ~safe_fit_mgcv(.x), .options = furrr_options(seed = T))

Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smo

In [14]:
# function to drop models that error
filter_errored_models <- function(models){
    models_f <- c()
    models_error <-c()
    for (m in names(models)){
        model <- models[[m]]
        if (is.null(model$error)){
            models_f[[m]] <- model$result
        }
        else {
            models_error[[m]] <- model
        }
    }
    return (models_f)
}



In [15]:
# filter down models
models <- filter_errored_models(models)
length(models)

[1] 304

In [16]:
# lets see what models look like
models[1]

$`chow_fast-HPF__01`

Family: gaussian 
Link function: identity 

Formula:
ME ~ get(group_col)
Total model degrees of freedom 2 

REML score: -3.955344     


In [17]:
# summarize models
plan(multisession, workers = 20)
summaries <- future_map(models, ~summary(.x))

In [18]:
length(summaries)

[1] 304

In [19]:
# make a vector of summaries
mgcv_summaries_f <- c()
for (s in names(summaries)){
    summary <- summaries[[s]]
    p.pv <- summary$p.pv[[2]]
    mgcv_summaries_f[[s]] <-summary
}

In [20]:
# save vector just in case
saveRDS(mgcv_summaries_f, summaries_path) 

In [21]:
# save models
saveRDS(models, models_path)

In [22]:
# data structure for export
ym_export <- c()
for (n in names(mgcv_summaries_f)){
    s <- mgcv_summaries_f[[n]]
    pval <- s$p.pv[2]
    coef <- s$p.coeff[2]
    ym_export[[n]] <- list('pvalue' = pval, "coef" = coef)
}

In [23]:
# what does it look like
ym_export

$`chow_fast-HPF__01`
$`chow_fast-HPF__01`$pvalue
get(group_col)fast 
         0.6222822 

$`chow_fast-HPF__01`$coef
get(group_col)fast 
       -0.04059185 


$`chow_fast-HPF__02`
$`chow_fast-HPF__02`$pvalue
get(group_col)fast 
         0.3572571 

$`chow_fast-HPF__02`$coef
get(group_col)fast 
        0.09558561 


$`chow_fast-HPF__03`
$`chow_fast-HPF__03`$pvalue
get(group_col)fast 
         0.1907902 

$`chow_fast-HPF__03`$coef
get(group_col)fast 
        0.03185054 


$`chow_fast-HPF__04`
$`chow_fast-HPF__04`$pvalue
get(group_col)fast 
         0.8279528 

$`chow_fast-HPF__04`$coef
get(group_col)fast 
       -0.01743792 


$`chow_fast-HPF__05`
$`chow_fast-HPF__05`$pvalue
get(group_col)fast 
        0.08960888 

$`chow_fast-HPF__05`$coef
get(group_col)fast 
         0.1301847 


$`chow_fast-HPF__06`
$`chow_fast-HPF__06`$pvalue
get(group_col)fast 
         0.8966645 

$`chow_fast-HPF__06`$coef
get(group_col)fast 
      -0.006745312 


$`chow_fast-HPF__07`
$`chow_fast-HPF__07`$pvalue
get(group_col)fast 
         0.5695811 

$`chow_fast-HPF__07`$coef
get(group_col)fast 
       -0.02513145 


$`chow_fast-HPF__08`
$`chow_fast-HPF__08`$pvalue
get(group_col)fast 
         0.6411432 

$`chow_fast-HPF__08`$coef
get(group_col)fast 
       -0.05030854 


$`chow_fast-HPF__09`
$`chow_fast-HPF__09`$pvalue
get(group_col)fast 
         0.5317764 

$`chow_fast-HPF__09`$coef
get(group_col)fast 
        0.03953805 


$`chow_fast-HPF__10`
$`chow_fast-HPF__10`$pvalue
get(group_col)fast 
         0.5815772 

$`chow_fast-HPF__10`$coef
get(group_col)fast 
        -0.0849058 


$`chow_fast-HPF__11`
$`chow_fast-HPF__11`$pvalue
get(group_col)fast 
        0.05022004 

$`chow_fast-HPF__11`$coef
get(group_col)fast 
        -0.1582766 


$`chow_fast-HPF__12`
$`chow_fast-HPF__12`$pvalue
get(group_col)fast 
        0.03516842 

$`chow_fast-HPF__12`$coef
get(group_col)fast 
        -0.1907627 


$`chow_fast-HPF__13`
$`chow_fast-HPF__13`$pvalue
get(group_col)fast 
         0.2960976 

$`chow_fast-HPF__13`$coef
get(group_col)fast 
        0.03757696 


$`chow_fast-HPF__14`
$`chow_fast-HPF__14`$pvalue
get(group_col)fast 
         0.2293215 

$`chow_fast-HPF__14`$coef
get(group_col)fast 
         0.1123299 


$`chow_fast-HPF__15`
$`chow_fast-HPF__15`$pvalue
get(group_col)fast 
         0.7946458 

$`chow_fast-HPF__15`$coef
get(group_col)fast 
        0.01659048 


$`chow_fast-HPF__16`
$`chow_fast-HPF__16`$pvalue
get(group_col)fast 
         0.4879808 

$`chow_fast-HPF__16`$coef
get(group_col)fast 
       -0.04747908 


$`chow_fast-HPF__17`
$`chow_fast-HPF__17`$pvalue
get(group_col)fast 
         0.5563658 

$`chow_fast-HPF__17`$coef
get(group_col)fast 
       -0.01145683 


$`chow_fast-HPF__18`
$`chow_fast-HPF__18`$pvalue
get(group_col)fast 
         0.6853056 

$`chow_fast-HPF__18`$coef
get(group_col)fast 
       -0.04896207 


$`chow_fast-HPF__19`
$`chow_fast-HPF__19`$pvalue
get(group_col)fast 
         0.3294426 

$`chow_fast-HPF__19`$coef
get(group_col)fast 
         0.0308794 


$`chow_fast-HPF__20`
$`chow_fast-HPF__20`$pvalue
get(group_col)fast 
         0.8187146 

$`chow_fast-HPF__20`$coef
get(group_col)fast 
      -0.008554401 


$`chow_fast-HPF__21`
$`chow_fast-HPF__21`$pvalue
get(group_col)fast 
          0.799502 

$`chow_fast-HPF__21`$coef
get(group_col)fast 
       -0.01268629 


$`chow_fast-HPF__22`
$`chow_fast-HPF__22`$pvalue
get(group_col)fast 
         0.2753177 

$`chow_fast-HPF__22`$coef
get(group_col)fast 
        0.06320383 


$`chow_fast-HPF__23`
$`chow_fast-HPF__23`$pvalue
get(group_col)fast 
         0.3081543 

$`chow_fast-HPF__23`$coef
get(group_col)fast 
        0.07986156 


$`chow_fast-HPF__24`
$`chow_fast-HPF__24`$pvalue
get(group_col)fast 
        0.08404223 

$`chow_fast-HPF__24`$coef
get(group_col)fast 
       -0.04787939 


$`chow_fast-HPF__25`
$`chow_fast-HPF__25`$pvalue
get(group_col)fast 
        0.04211365 

$`chow_fast-HPF__25`$coef
get(group_col)fast 
       -0.08966411 


$`chow_fast-HPF__26`
$`chow_fast-HPF__26`$pvalue
g

In [24]:
# convert to tibble and save as csv
df <- ym_export %>%
  imap(function(value, name) {
    tibble(
      test_name = name,
      coef = value$coef,
      pvalue = value$pvalue
    )
  }) %>%
  bind_rows() %>% 
  arrange(pvalue) %>% 
  mutate(pvalue_bh = p.adjust(pvalue, method = "BH")) %>% 
  arrange(pvalue_bh) %>%
  separate('test_name', into = c('comparison', 'module'), sep='-') %>%
  mutate(comparison = str_replace(comparison, fixed('_'), '.vs.'))

df %>% head(10)
df %>% write_csv(significant_modules)

comparison,module,coef,pvalue,pvalue_bh
<chr>,<chr>,<dbl>,<dbl>,<dbl>
chow.vs.fast,HY__50,0.20788818,4.138570e-06,0.001258125
chow.vs.fast,HY__71,-0.15160800,5.928408e-05,0.009011179
chow.vs.fast,IC__15,0.18669571,1.400258e-04,0.013869961
chow.vs.fast,IC__32,-0.17682802,2.418786e-04,0.013869961
chow.vs.fast,SC__78,0.18925718,2.422843e-04,0.013869961
chow.vs.fast,HPF__33,0.12701618,2.737492e-04,0.013869961
chow.vs.fast,IC__07,-0.14618702,6.374605e-04,0.027684000
chow.vs.fast,IC__08,0.14977742,1.321558e-03,0.049536016
chow.vs.fast,SC__32,-0.17492720,1.466527e-03,0.049536016


In [25]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux

Matrix products: default
BLAS/LAPACK: /nfsdata/tools/anaconda/envs/nmq407/dmb/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_GB.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_GB.UTF-8        LC_COLLATE=en_GB.UTF-8    
 [5] LC_MONETARY=en_GB.UTF-8    LC_MESSAGES=en_GB.UTF-8   
 [7] LC_PAPER=en_GB.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_GB.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] lubridate_1.9.2             stringr_1.5.0              
 [3] purrr_1.0.1                 readr_2.1.4                
 [5] tibble_3.2.1                tidyverse_2.0.0            
 [7] furrr_0.3.1                 future_1.32.0              
 [9] fitdistrplus_1.1-11         sur